In [7]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/aditi/nltk_data...


True

In [9]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
37,It was very refreshing to watch this beautiful...,positive
117,First off I am in my mid 40's. Been watchin ho...,negative
8,This movie is silly and very short of being a ...,negative
474,I really don't understand who this movie is ai...,negative
920,mature intelligent and highly charged melodram...,positive


In [10]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [11]:
df = normalize_text(df)
df.head()

,review,sentiment
37,refreshing watch beautiful movie director main...,positive
117,first mid s watchin horror film since kid seen...,negative
8,movie silly short funny movie unhappy easterne...,negative
474,really understand movie aimed at absurdity it ...,negative
920,mature intelligent highly charged melodrama un...,positive


In [12]:
df['sentiment'].value_counts()

sentiment
negative    259
positive    241
Name: count, dtype: int64

In [13]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [14]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
37,refreshing watch beautiful movie director main...,1
117,first mid s watchin horror film since kid seen...,0
8,movie silly short funny movie unhappy easterne...,0
474,really understand movie aimed at absurdity it ...,0
920,mature intelligent highly charged melodrama un...,1


In [15]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [16]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/aditivj/capstone-project.mlflow')
dagshub.init(repo_owner='aditivj', repo_name='capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as aditivj

Initialized MLflow to track repo "aditivj/capstone-project"

Repository aditivj/capstone-project initialized!

2025/12/28 23:24:43 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/78855312c49c43ccbedb80fc85e56139', creation_time=1766944483522, experiment_id='0', last_update_time=1766944483522, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 75)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-12-28 23:31:34,963 - INFO - Starting MLflow run...
2025-12-28 23:31:35,417 - INFO - Logging preprocessing parameters...
2025-12-28 23:31:36,537 - INFO - Initializing Logistic Regression model...
2025-12-28 23:31:36,538 - INFO - Fitting the model...
2025-12-28 23:31:36,574 - INFO - Model training complete.
2025-12-28 23:31:36,575 - INFO - Logging model parameters...
2025-12-28 23:31:36,910 - INFO - Making predictions...
2025-12-28 23:31:36,912 - INFO - Calculating evaluation metrics...
2025-12-28 23:31:36,919 - INFO - Logging evaluation metrics...
2025-12-28 23:31:38,280 - INFO - Saving and logging the model...
2025/12/28 23:31:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-28 23:31:49,447 - INFO - Model training and logging completed in 14.03 seconds.
2025-12-28 23:31:49,458 - INFO - Accuracy: 0.648
2025-12-28 23:31:49,459 - INFO - Precision: 0.6666666666666666
2025-12-28 23:31:49,460 - INFO - Recall: 0.625
2025-12-28 23:31:49,460

🏃 View run vaunted-pig-580 at: https://dagshub.com/aditivj/capstone-project.mlflow/#/experiments/0/runs/a9da0e6819804fd999825bc946170cf3
🧪 View experiment at: https://dagshub.com/aditivj/capstone-project.mlflow/#/experiments/0
